### 04 ~ 07 예제파일 통합

In [1]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import tensorflow as tf

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

Using TensorFlow backend.


In [2]:
# 데이터 입력
df = pd.read_csv('../dataset/sonar.csv', header=None)

# 데이터 개괄 보기
# print(df.info())

# 데이터의 일부분 미리 보기
# print(df.head())

print('shape:',df.shape)
df.head()

shape: (208, 61)


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

print(np.unique(Y))

[0 1]


In [4]:
# 모델 설정
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['accuracy'])

# 모델 실행 : 표시안함
model.fit(X, Y, epochs=100, batch_size=10, verbose=0)

# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

208/208 [==============================] - 0s 106us/step

 Accuracy: 0.9567


In [5]:
from sklearn.model_selection import train_test_split

# 학습 셋과 테스트 셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)
print('shape:',X_train.shape, X_test.shape)

shape: (145, 60) (63, 60)


In [6]:
# 모델 재 설정
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=100, batch_size=10, verbose=0)

# 테스트셋에 모델 적용
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

63/63 [==============================] - 0s 557us/step

 Test Accuracy: 0.7937


In [7]:
# 모델을 컴퓨터에 저장
model.save('my_model.h5')

In [8]:
from keras.models import load_model

# 테스트를 위해 메모리 내의 모델을 삭제
del model

# 모델 불러옴
model = load_model('my_model.h5') 

# 불러온 모델로 테스트 실행
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))  

63/63 [==============================] - 0s 771us/step

 Test Accuracy: 0.7937


In [9]:
from sklearn.model_selection import StratifiedKFold

# 10개의 파일로 쪼갬
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [10]:
# 빈 accuracy 배열
accuracy = []

# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=10, verbose=0)
#     k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
#     accuracy.append(k_accuracy)
    accuracy.append( model.evaluate(X[test], Y[test])[1] )

    
# 결과 출력
# print("\n %.f fold accuracy: " % n_fold, accuracy)
print()
print("{0} fold accuracy average: {1:.4f}".format(n_fold, np.average(accuracy)))
print('accuracies: ',', '.join('{:.4f}'.format(a) for a in accuracy))

20/20 [==============================] - 0s 7ms/step

10 fold accuracy average: 0.8189
accuracies:  0.6364, 0.8571, 0.8571, 0.9048, 0.7619, 0.6667, 0.9048, 0.8000, 0.9000, 0.9000
